In [1]:
import pickle
import numpy as np
import pandas as pd

### Distances

In [31]:
from similarity.weighted_levenshtein import WeightedLevenshtein
from similarity.weighted_levenshtein import CharacterSubstitutionInterface

In [32]:
def wup_seq(c2c, v1, v2):
    all_icd9 = sorted(list(set(v1 + v2)))
    symbols = [chr(i) for i in range(len(all_icd9))]  # fix this
    coded_all = [symbols[i] for i in range(len(all_icd9))]
    coded_v1 = ''.join([coded_all[all_icd9.index(c)] for c in sorted(v1)])
    coded_v2 = ''.join([coded_all[all_icd9.index(c)] for c in sorted(v2)])
    encoder = {k: v for (k, v) in zip(all_icd9, coded_all)}
    decoder = {v: k for (k, v) in zip(all_icd9, coded_all)}

    encoded_c2c = {}
    for icd9_1 in v1:
        for icd9_2 in v2:
            encoded_c2c[(encoder[icd9_1], encoder[icd9_2])] = 1 - c2c[(icd9_1, icd9_2)]

    class CharacterSubstitution(CharacterSubstitutionInterface):
        def cost(self, c0, c1):
            return encoded_c2c[(c0, c1)]

    weighted_levenshtein = WeightedLevenshtein(CharacterSubstitution())
    return weighted_levenshtein.distance(coded_v1, coded_v2)

def wup_points(c2c, s, t):
    n = len(s)
    m = len(t)
    dtw = np.full((n, m), 10000, dtype=np.float64)
    dtw[0, 0] = 0
    for i in range(n):
        for j in range(m):
            cost = np.round(wup_seq(c2c, s[i], t[j]), 3)
            in_cost = dtw[i - 1, j] if i > 0 else 0
            del_cost = dtw[i, j - 1] if j > 0 else 0
            edit_cost = dtw[i - 1, j - 1] if i > 0 and j > 0 else 0
            dtw[i, j] = cost + min(in_cost, del_cost, edit_cost)
    return dtw[-1][-1]

def wup_concepts(a1, a2):
    """
    This function calculates the WUP distance between concept C1 and concept C2 in the ontology:

    WUP(C1,C2) = (2*depth_LCA)/(N_1+N_2+depth_LCA)

    where depth_LCA is the depth of the last common ancestor (LCA) of concept1 and concept2
    N_1 is the number of nodes from LCA to concept C1
    N_2 is the number of nodes from LCA to concept C2

    :param a1: path from ROOT to the concept C1
    :param a2: path from ROOT to the concept C2
    :return:
    """
    # common substring length -> depth of last common ancestor (LCA)
    # a1, a2 are PATHS FROM ROOT

    depth_LCA = 0
    while True:
        if depth_LCA >= len(a1) or depth_LCA >= len(a2) or a1[depth_LCA] != a2[depth_LCA]:
            break
        depth_LCA += 1

    dr = depth_LCA - 1
    da = len(a1) - depth_LCA
    db = len(a2) - depth_LCA

    return ((2 * dr) / (da + db + 2 * dr))

### Utils

In [16]:
import networkx as nx
import csv

In [25]:
def pickle_save(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [26]:
def read_CSV_ontology(ontology_path_file):
    """
    :param ontology_path_file: the path to the ontology file with the name  of the CSV file
    e.g. "/path/to/file/ICD9_ontology.csv"
    :return: a networkx Directed Graph
    """
    ont_tree = nx.DiGraph()
    nodes = set()
    line_count = 0
    with open(ontology_path_file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            line_count += 1
            ont_tree.add_edge(row[1], row[0])
            nodes.add(row[1])
            nodes.add(row[0])
        print(f'The ontology has {line_count} "is-a" links.')
        print(f'The ontology has {len(nodes)} nodes')

    return ont_tree

def subset_ontology_from_data(ont_tree, dataset_sequences):
    """
    This function selects the subset of concepts from the original ontology that also appears in the data
    :param ont_tree: nx.DiGraph, original ontology
    :param dataset_sequences: list of lists of lists, the sequences of the dataset
    :return: nx.DiGraph
    """

    data_concepts = sorted(list(set([a for b in [c for d in dataset_sequences for c in d] for a in b])))
    data_nodes = []

    for l in data_concepts:
        data_nodes += nx.shortest_path(ont_tree, 'ROOT', l)
    data_nodes = sorted(list(set(data_nodes)))

    # PARENT->CHILD
    subset_ontology = nx.DiGraph()
    for child in data_nodes:
        if child == 'ROOT':
            continue
        parent = list(ont_tree.in_edges(child))[0][0]
        subset_ontology.add_edge(parent, child)

    return subset_ontology

In [27]:
def compute_c2c(ont_tree, dataset, output_path, verbose=True):
    """
    This function computes and save a python dictionary containing all pairwise distances between the concepts of the ontology present in the dataset used to train and test the blackbox.

    :param ont_tree: nx.DiGraph, is the ontology
    :param dataset: list of lists of lists, is the dataset used to train and test the blackbox
    :param output_path: str, is the path where you want to save the output
    :return: python dictionary, with key the tuple of concepts and value the pairwise distances between the concepts
    """

    data_concepts = sorted(list(set([a for b in [c for d in dataset for c in d] for a in b])))
    subset_ontology = subset_ontology_from_data(ont_tree, dataset)
    paths_from_ROOT = [nx.shortest_path(subset_ontology, 'ROOT', c) for c in data_concepts]
    n_paths = len(paths_from_ROOT)

    c2c_dict = {}

    for i, v1 in enumerate(paths_from_ROOT):
        if verbose:
            if i % 100 == 0:
                print(f'{i} concepts elaborated out of {n_paths}')
        for v2 in paths_from_ROOT:
            c2c_dict[(v1[-1], v2[-1])] = wup_concepts(v1, v2)
    if verbose:
        print(f'{i} concepts elaborated')
    pickle_save(c2c_dict, f'{output_path}/c2c_dist')

    return c2c_dict

### Real neighbors identification

In [3]:
#these are the pickle dictionaries containing the descriptions of the codes (ICD-9 and CCS codes)
ICD9_description_dict = pickle.load(open('../ds/ICD9_description_dict.pkl', 'rb'))
CCS_description_dict = pickle.load(open('../ds/CCS_description_dict.pkl', 'rb'))

In [8]:
dataset = np.load('../ds/preprocessing_doctorai/mimic_sequences.npy',allow_pickle=True)
datapoint = dataset[0]
ontology_path = '../ds/ICD9_ontology.csv'

In [18]:
ont_tree = read_CSV_ontology(ontology_path)

The ontology has 22535 "is-a" links.
The ontology has 22536 nodes


In [30]:
c2c_distance_matrix = compute_c2c(ont_tree, dataset, '.')

0 concepts elaborated out of 259
100 concepts elaborated out of 259
200 concepts elaborated out of 259
258 concepts elaborated


In [44]:
def find_closest_neighbors(datapoint, dataset, c2c_distance_matrix, n_first_neighbors = 10):
        """
        This function finds the first k (n_first_neighbors) closest neighbors of the analyzed patient (patient_sequence)
        in the dataset (dataset_sequences). It does so by using the ontology.
        :return: list of lists of lists, closest neighbours patients' sequences including the patient under analysis
        """

        if n_first_neighbors<0:
            print('n_first_neighbors should be a positive integer or 0')
        if n_first_neighbors==0:
            print('Perturbing only the patients to be explained')
            real_neighs = [datapoint]
        else:
            #calculate the k closest neighbors in the dataset
            c2c_dict = c2c_distance_matrix
            points_dist = {}
            
            for i, point in enumerate(dataset):
                points_dist[i] = wup_points(c2c_dict, datapoint, point)
            
            closest_neigh_indexes = list(
                    {k: v for k, v in 
                    sorted(points_dist.items(), key=lambda item: item[1])
                    }.keys())[:n_first_neighbors+1]
            real_neighs = [dataset[idx] for idx in closest_neigh_indexes]

        return real_neighs

In [46]:
find_closest_neighbors(datapoint, dataset, c2c_distance_matrix, 20)[0]

[['571.5',
  '584.9',
  '070.70',
  '280.0',
  '287.5',
  '789.5',
  '456.20',
  '572.3',
  '401.9'],
 ['571.5',
  '452',
  '070.70',
  '280.0',
  '456.20',
  '038.9',
  '995.92',
  '785.52',
  '518.81',
  '572.4',
  '584.9',
  '585.9',
  '789.5',
  '572.3']]